In [2]:
import os
import requests
from pyspark.sql import SparkSession
from tmlt.analytics.privacy_budget import PureDPBudget
from tmlt.analytics.query_builder import QueryBuilder
from tmlt.analytics.session import Session

r = requests.get(
    'https://tumult-public.s3.amazonaws.com/library-members.csv',
)
with open("members.csv", "w") as f:
    f.write(r.text)
spark = SparkSession.builder.getOrCreate()
members_df = spark.read.csv("members.csv", header=True, inferSchema=True)

In [4]:
session = Session.from_dataframe(
    privacy_budget=PureDPBudget(epsilon=float('inf')),
    source_id="members",
    dataframe=members_df,
)

In [9]:
mean_age_query = QueryBuilder("members").average("age", low=0, high=200)
mean_age = session.evaluate(
    mean_age_query,
    privacy_budget=PureDPBudget(epsilon=10)
)
mean_age.show()

+-----------------+
|      age_average|
+-----------------+
|38.04638766438571|
+-----------------+



In [7]:
itms_list = members_df.select('id').collect()

In [ ]:
itms_array = [int(row.id) for row in itms_list]

In [8]:
from tmlt.analytics.keyset import KeySet

edu_levels = KeySet.from_dict({
    "id": itms_array
})

NameError: name 'itms_array' is not defined

In [27]:
len(itms_array)

54217

In [ ]:
edu_average_age_query = (
    QueryBuilder("members")
    .groupby(edu_levels)
    .average("age", low=0, high=120)
)
edu_average_ages = session.evaluate(
    edu_average_age_query,
    privacy_budget=PureDPBudget(100),
)
edu_average_ages.sort("age_average").show(truncate=False)